# Rotor-Design Utilities

## Planform constraints
Demo showing how to apply planform constraints, such as: 

- `root_chord`: Returns a smooth transition from an ideal aerodynamic chord that satisfy a root-chord- and a maximum-chord-size.
- `min_tc_chord`: Returns a chord that maintains the minimum $t/c$ for the tip region (it is often the case that the tip-loss model leads to an increase in $t/c$ in this region)
- `max_twist`: Returns a twist that satisfy a maximum twist

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lacbox.rotor_design.utils import root_chord, min_tc_chord, max_twist
from lacbox.test import test_data_path

# Load data
r, t, chord_ideal, tc_ideal, twist_ideal = np.loadtxt(test_data_path+"/ideal_planform.dat", unpack=True)

### Root chord transition (`root_chord`)
Demo showing how to make a chord design at the root that satisfy a given chord size at the root and a maximum chord from a ideal aerodynamic designed chord.

In [ ]:
# Apply the root chord transition
chord_root = t[0]
chord_max = 3
chord_rc = root_chord(r, chord_ideal, chord_root, chord_max)
# Calculating new t/c
tc_rc = t/chord_rc*100

# Plot the ideal- and root-chord transition constraint chord and t/c
# Chord
fig, axs = plt.subplots(2)
ax = axs[0]
ax.plot(r, chord_ideal, label="Ideal")
ax.plot(r, chord_rc, label="Root chord transition")
ax.set_ylabel("Chord [m]")
ax.legend()
# t/c
ax = axs[1]
ax.plot(r, tc_ideal)
ax.plot(r, tc_rc)
ax.set_xlabel("Span [m]")
ax.set_ylabel("t/c [%]")
fig.tight_layout()

### Constrain t/c to be decreasing (`min_tc_chord`)
Demo showing how to get a monotonically decreasing relative profile thickness ($t/c$) along the span.

In [ ]:
# Applying the constraint on t/c
chord = min_tc_chord(chord_rc, t)
# Computing the new t/c
tc = t/chord*100

# Plot the ideal- and constraint chord and t/c
# Chord
fig, axs = plt.subplots(2)
ax = axs[0]
ax.plot(r, chord_rc, label="Root chord transition")
ax.plot(r, chord, label="Decreasing t/c")
#ax.set_xlabel("Span [m]")
ax.set_ylabel("Chord [m]")
ax.legend()
# t/c
ax = axs[1]
ax.plot(r, tc_rc)
ax.plot(r, tc)
ax.set_xlabel("Span [m]")
ax.set_ylabel("t/c [%]")
fig.tight_layout()

### Limit the maximum twist angel (`max_twist`)
Demo showing how to apply a maximum twist constraint for an ideal aerodynamic twist.

In [ ]:
# Input
twist_max = 20

# Applying the maximum twist
twist = max_twist(twist_ideal, twist_max)

# Plotting the ideal and constraint twist
fig, ax = plt.subplots()
ax.axhline(twist_max, ls="--", lw=1, color="k", label="Maximum twist")
ax.plot(r, twist_ideal, label="Ideal")
ax.plot(r, twist, label="With maximum twist")
ax.set_xlabel("Span [m]")
ax.set_ylabel("Twist [deg]")
ax.legend()
fig.tight_layout()

## Aerodynamic polar design interpolator (`interpolator`)
Demo showing how to use a set of aerodynamic polar design points as a function of the profile thickness ($t/c$).

In [ ]:
# Import the interpolator
from lacbox.rotor_design.utils import interpolator
import numpy as np
import matplotlib.pyplot as plt

# Design value (extended to have a valid range from 0-100 including zero tangent at the ends)
tc_des_vals = [0, 15, 18, 24, 30, 36, 100, 105]
cl_des_vals = [0.9, 0.9, 0.8, 0.8, 0.7, 0.6, 0.0, 0.0]
cd_des_vals = [0.00850, 0.00850, 0.00604, 0.0107, 0.0139, 0.0155, 0.5, 0.5]
aoa_des_vals = [5.0, 5.0, 4.3, 4.3, 4.0, 0.5, 0.0, 0.0]

# Creating the interpolator function
cl_des = interpolator(tc_des_vals, cl_des_vals)
cd_des = interpolator(tc_des_vals, cd_des_vals)
aoa_des = interpolator(tc_des_vals, aoa_des_vals)

# Plotting the design functions
tc = np.linspace(0, 100, 100)
fig, axs = plt.subplots(3, 1)
# Cl
ax = axs[0]
ax.plot(tc, cl_des(tc), "k")
ax.plot(tc_des_vals[1:-1], cl_des_vals[1:-1], "ok")
ax.set_ylabel("$C_{l,design}$ [-]")
# Cd
ax = axs[1]
ax.plot(tc, cd_des(tc), "k", label="Design function")
ax.plot(tc_des_vals[1:-1], cd_des_vals[1:-1], "ok", label="Design values")
ax.set_ylabel("$C_{d,design}$ [-]")
ax.legend()
# AoA
ax = axs[2]
ax.plot(tc, aoa_des(tc), "k")
ax.plot(tc_des_vals[1:-1], aoa_des_vals[1:-1], "ok")
ax.set_xlabel("Relative profile thickness (t/c) [%]")
ax.set_ylabel(r"AoA ($\alpha$) [deg]")
fig.tight_layout()